# 栄養パターンに基づく口腔・上部消化管がん分類モデルの構築および評価2

# 概要

## パッケージダウンロード

In [ ]:
rm(list = ls())

In [ ]:
install.packages("MatchIt")
install.packages("cobalt")
install.packages("dplyr")
install.packages("ggplot2")
install.packages("tidyverse")
library(MatchIt)
library(cobalt)
library(dplyr)
library(ggplot2)
library(tidyverse)

## データの読み込み

In [ ]:
# データフレームを読み込む
file_path <- "merged_df5.csv"

# read.csv() 関数を使用してCSVファイルを読み込む
# header = TRUE: 1行目を列名として使用
# stringsAsFactors = FALSE: 文字列を自動で因子型（Factor）に変換しないようにする
merged_df7 <- read.csv(file_path, header = TRUE, stringsAsFactors = FALSE)

# 読み込みが成功したか確認
print("データフレームの最初の数行:")
head(merged_df7)

print("データフレームの構造（変数型）:")
str(merged_df7)

## PSM処理

### TargetNameの調節

In [ ]:
# 'mydata' データフレームの 'diagnosis' 列を変換
merged_df7 <- merged_df7 %>%
  mutate(TargetID = if_else(oral == 1, 1, 
                                    if_else(oral == 0, 0, NA_real_))) # NA_real_ は欠損値として扱う
# 結果の確認
head(merged_df7)

In [ ]:
merged_df7 <- merged_df7 %>%
  mutate(TargetName = if_else(oral == 1, "oral_cancer", 
                                    if_else(oral == 0, "non_cancer",NA_character_))) # NA_real_ は欠損値として扱う
head(merged_df7)

In [ ]:
# 含まれているユニークな値のリストを取得
unique(merged_df7$TargetName)

###　データフレーム抽出

今回必要なデータフレームの列名は、sex, site, age, BMI(height_q,weight_q),dr_yn,sm_yn,

In [ ]:
missing_values_count <- colSums(is.na(merged_df7))

# 3. 結果の表示
print("---------------------------")
print("各列の欠損値の数:")
print(missing_values_count)

### 欠損値処理およびデータ型変更

In [ ]:
# 'TargetID'は処置変数、残りは交絡変数
cols_for_psm <- c("sex","site","educ","age","bmiq","methw","dr_yn","sm_yn","TargetID")

# 2. 指定した列（cols_for_psm）に欠損値がない行だけを抽出して、新しいデータフレームを作成
merged_df7_cln <- merged_df7[complete.cases(merged_df7[, cols_for_psm]), ]

# 処理結果の確認
print(paste("元の行数:", nrow(merged_df7)))
print(paste("欠損値を除外した後の行数:", nrow(merged_df7_cln)))

In [ ]:
merged_df7_cln$C_edu <- as.character(merged_df7_cln$edu)

In [ ]:
merged_df7_cln$C_dr_yn <- as.character(merged_df7_cln$dr_yn)

In [ ]:
merged_df7_cln$C_site<- as.character(merged_df7_cln$site)

In [ ]:
merged_df7_cln$C_sex<- as.character(merged_df7_cln$sex)

In [ ]:
merged_df7_cln$C_sm_yn <- as.character(merged_df7_cln$sm_yn)

In [ ]:
# データフレームのデータ方を確認
str(merged_df7_cln)

### PSM実行

In [ ]:
# PSMの実行: ロジスティック回帰でPSを推定し、1:1の最近傍マッチング（非復元抽出）を行う
m.out <- matchit(TargetID ~ C_sex + C_site + age + C_edu + C_dr_yn + C_sm_yn + bmiq + methw,
                 data = merged_df7_cln,
                 method = "nearest", 
                 distance = "glm",  # ロジスティック回帰を使用
                 ratio = 1,         # 1:1マッチング
                 replace = FALSE)   # 非復元抽出 (一度マッチングに使用された対照群は再使用しない)

# マッチング結果の概要を確認
summary(m.out, standardize = TRUE)

In [ ]:
# `cobalt`パッケージの`bal.tab()`関数を使用
bal.tab(m.out, un = TRUE, stats = c("m", "sd", "v.ratio"))

### 結果出力

In [ ]:
# 1. matchitオブジェクトから、マッチング後のデータフレームを抽出する
matched_data <- match.data(m.out)

# 2. 抽出したデータフレームをCSVとして保存する
write.csv(
  x = matched_data,
  file = "merged_R.csv",
  row.names = FALSE,
  na = ""
)

In [ ]:
# マッチング前後の傾向スコアの分布図
plot(m.out, type = "hist")
# 共変量のバランスプロット
plot(m.out, type = "qq")

## PSM処理前後の変化図

In [ ]:
# マッチング後のデータフレームを抽出
matched_data <- match.data(m.out)

# 抽出されたデータフレームの確認
head(matched_data)

In [ ]:
# 抽出したい列名 (ベクトルとして定義)
selected_columns <- c("TargetID", "C_sex", "site","C_edu","C_dr_yn","C_sm_yn","age", "bmiq", "methw")

# 元のデータフレームが 'merged_df7' だと仮定します
extracted_df <- merged_df7_cln[ , selected_columns]

# 結果の確認 (最初の数行と列名を表示)
head(extracted_df)
colnames(extracted_df)
# MatchItの結果をデータフレームとして取得 (マッチング前後の重み情報を含む)
m_data <- match.data(m.out, data = extracted_df)

# 欠損値を含む行を落とした元のデータ (比較のため)
# is.na()で重みがNAの行を落とします。
m_data_unmatched <- merged_df7_cln %>% 
  drop_na(
    TargetID, C_sex, site, age, bmiq, methw,  # 数値・二値変数
    C_edu,     # 1. 元のカテゴリ変数名
    C_dr_yn,   # 2. 元のカテゴリ変数名
    C_sm_yn    # 3. 元のカテゴリ変数名
  ) %>%
  mutate(weights = 1, subgroup = "Unmatched")# 全員に重み1を設定

# マッチング後のデータにサブグループラベルを追加
m_data_matched <- m_data %>% 
  mutate(subgroup = "Matched")

# 比較のために2つのデータセットを結合 (rbind)
plot_data <- bind_rows(m_data_unmatched, m_data_matched)

In [ ]:
covariates_to_plot <- c("age", "C_sex", "C_site", "C_edu", "C_dr_yn", "C_sm_yn", "bmiq", "methw")

for (var_name in covariates_to_plot) {
  
  # --- 2-1. プロットのベースを作成 ---
  p <- ggplot(plot_data, aes(x = .data[[var_name]], fill = factor(TargetID), weight = weights)) +
    
    # facet_wrapで Before/After を分割
    facet_wrap(~subgroup, scales = "free_y", ncol = 1) + 
    
    # 軸ラベルの設定
    labs(
      title = paste(var_name, "Distribution Before and After Matching"),
      x = var_name,
      y = "Frequency (Weighted)",
      fill = "Group (0 = non_cancer, 1 = cancer)"
    ) +
    theme_minimal()
    
  # 'plot_data'内での変数の型を確認
  var_type <- class(plot_data[[var_name]])
  
  if (var_type %in% c("numeric", "integer", "double")) {
    # age, bmiq, methw のような数値変数: ヒストグラム
    p <- p + geom_histogram(bins = 30, position = "identity", alpha = 0.6)
  } else if (var_type %in% c("factor", "character")) {
    # sex, site, C_edu などのカテゴリ変数: 棒グラフ (count/frequency)
    p <- p + geom_bar(position = "dodge", alpha = 0.7) # position="dodge"で横に並べて比較
  } else {
    # その他の型の変数はスキップまたはエラーメッセージ
    message(paste("Warning: Skipping plot for unknown type of variable:", var_name, "(Type:", var_type, ")"))
    next # 次のループへ
  }
  
  # --- 2-3. プロットを表示/保存 ---
  print(p)
}

In [ ]:
# 結合した図を表示
print(combined_plot)

# ファイルに保存する際に、全体のサイズを調整
ggsave(
  filename = "Combined_Covariate_Balance_Plots.png",
  plot = combined_plot,                               
  width = 24,                                         
  height = 20,                                       
  units = "in"                                        
)

In [ ]:
plot(m.out, type = "jitter", interactive = FALSE)

In [ ]:
# 1. 保存するファイル名とファイルパスを指定
file_path <- "psm_jitter_plot_balance.png"

# 2. PNGグラフィックデバイスを開始
png(filename = file_path, width = 800, height = 600, units = "px", res = 100)

# 3. プロットコマンドを実行
plot(m.out, type = "jitter", interactive = FALSE)

dev.off()

cat(paste0("Jitterプロットは '", file_path, "' に保存されました。✅\n"))

## SMD出力

In [ ]:
library(cobalt)
bal.tab(m.out, stats = c("m", "v", "ks", "p"), thresholds = c(m = 0.05), un = TRUE)
# PSM実行前のバランス（SMDとP値）を確認
bal.tab(TargetID ~ C_sex + C_site + age + C_edu + C_dr_yn + C_sm_yn + bmiq + methw,
        data = merged_df7_cln,
        stats = c("m", "v", "ks", "p"),
        thresholds = c(m = 0.05),
        un = TRUE) 

## 平均値　標準偏差　データ数　割合

In [ ]:
library(cobalt)

# bal.tabの出力結果
bal.out <- bal.tab(TargetID ~ age + bmiq + methw,
                   data = merged_df7_cln,
                   stats = c("m", "v", "ks", "p"),
                   thresholds = c(m = 0.05),
                   un = TRUE,
                   disp = c("means", "sds")) 
print(bal.out)

In [ ]:
library(cobalt)
library(dplyr) 
bal.out <- bal.tab(TargetID ~ age + bmiq + methw,
                   data = merged_df7_cln,
                   stats = c("m", "v", "ks", "p"),
                   thresholds = c(m = 0.05),
                   un = TRUE,
                   disp = c("means", "sds")) 

bal.out <- bal.tab(m.out, 
                   stats = c("m", "v", "ks", "p"),
                   thresholds = c(m = 0.05),
                   un = TRUE,
                   disp = c("means", "sds"),
                   data = merged_df7_cln)

# bal.out$Balance からバランス統計のデータフレームを取得
balance_df <- bal.out$Balance %>%
  # 変数名（行名）を新しい列として保持
  tibble::rownames_to_column("Variable") 

# 保存ファイル名を設定
file_name <- "psm_balance_table.csv"

# write.csvまたはreadr::write_csvで保存
write.csv(balance_df, 
          file = file_name, 
          row.names = FALSE,
          fileEncoding = "UTF-8")

# 保存場所の確認（オプション）
cat("バランス表が現在の作業ディレクトリに保存されました: ", file_name, "\n")
cat("現在の作業ディレクトリ: ", getwd(), "\n")

In [ ]:
library(cobalt)
library(dplyr)
library(tibble) 

bal.out <- bal.tab(m.out, 
                   data = merged_df7_cln, 
                   stats = c("m", "v", "ks", "p"),
                   thresholds = c(m = 0.05),
                   un = TRUE,
                   disp = c("means", "sds")) 

print(bal.out)

# bal.out$Balance からバランス統計のデータフレームを取得
balance_df <- bal.out$Balance %>%
  # 変数名（行名）を新しい列として保持
  rownames_to_column("Variable") 

# 保存ファイル名を設定
file_name <- "psm_balance_continuous_table.csv"

# write.csvで保存
write.csv(balance_df, 
          file = file_name, 
          row.names = FALSE, 
          fileEncoding = "UTF-8")

cat("連続変数のPSMバランス表が現在の作業ディレクトリに保存されました: ", file_name, "\n")
cat("現在の作業ディレクトリ: ", getwd(), "\n")

In [ ]:
library(dplyr)
library(tidyr)
library(stringr)

# PSM前後のカウントと割合を計算し整形する関数
calculate_counts <- function(data_df, time_label) {
  
  # 集計対象のカテゴリ変数リスト
  categorical_vars <- c("C_sex", "C_site", "C_edu", "C_dr_yn", "C_sm_yn")
  
  # TargetID, サブクラス, 重み（PSM後のために必要）を選択
  data_df %>%
    select(TargetID, all_of(categorical_vars), any_of(c("weights", "subclass"))) %>%
    
    # Wide形式からLong形式へ変換し、TargetIDとLevelでグループ化
    pivot_longer(cols = all_of(categorical_vars), names_to = "Variable", values_to = "Level") %>%
    group_by(Variable, Level, TargetID) %>%
    summarise(
      Count = n(), # 度数（カウント）
      .groups = 'drop_last'
    ) %>%
    
    # 割合の計算 (TargetIDグループ内の割合)
    mutate(
      Total_N = sum(Count), # TargetIDごとの合計N
      Percentage = round(Count / Total_N * 100, 1), 
      Count_Perc = str_glue("{Count} ({Percentage}%)")
    ) %>%
    ungroup() %>%
    
    # 最終的な表示形式に整形
    select(Variable, Level, TargetID, Count_Perc) %>%
    pivot_wider(names_from = TargetID, values_from = Count_Perc, names_prefix = "TargetID_") %>%
    rename(
      Control = TargetID_0, 
      Case = TargetID_1
    ) %>%
    mutate(Time = time_label)
}
# PSM前 (Unmatched) のカウント
counts_unmatched <- calculate_counts(merged_df7_cln, "PSM前 (Unmatched)")

# PSM後 (Matched) のカウント
counts_matched <- calculate_counts(matched_data, "PSM後 (Matched)")

# 結果を結合し、表示
categorical_vars_table <- bind_rows(counts_unmatched, counts_matched) %>%
  # 変数名、水準、時間を基準に並べ替え
  arrange(Variable, Level, Time)

print(categorical_vars_table)

In [ ]:
file_name_categorical <- "psm_categorical_table.csv"

# write.csvでデータフレームを保存
write.csv(categorical_vars_table, 
          file = file_name_categorical, 
          row.names = FALSE, 
          fileEncoding = "UTF-8")

cat("カテゴリ変数のバランス表が現在の作業ディレクトリに保存されました: ", file_name_categorical, "\n")
cat("現在の作業ディレクトリ: ", getwd(), "\n")

In [ ]:
library(cobalt)
library(dplyr)

bal_result_matched <- bal.tab(m.out, 
                              stats = c("m", "v", "ks", "p"), 
                              thresholds = c(m = 0.05), 
                              un = TRUE)

bal_result_unmatched <- bal.tab(TargetID ~ C_sex + C_site + age + C_edu + C_dr_yn + C_sm_yn + bmiq + methw,
                                data = merged_df7_cln,
                                stats = c("m", "v", "ks", "p"),
                                thresholds = c(m = 0.05),
                                un = TRUE)

# 例: マッチング後の結果（bal_result_matched）から表を抽出
balance_table_df <- bal_result_matched$Balance

# 抽出したデータフレーム（表）の表示
print("--- Extracted Balance Table (Data Frame) ---")
print(balance_table_df)


if ("Unadj.M.Diff" %in% names(balance_table_df)) {
    clean_table <- balance_table_df %>%
        select(
            Variable, 
            starts_with("Unadj.M"), 
            starts_with("Adj.M"),   
            matches("\\.p\\.")      
        )
    
    print("\n--- Cleaned Balance Table ---")
    print(clean_table)
}

In [ ]:
file_path <- "balance_statistics_matched.csv" 

write.csv(
    balance_table_df, 
    file = file_path, 
    row.names = FALSE,
    fileEncoding = "UTF-8" # 日本語環境での文字化けを防ぐため
)

print(paste0("バランス統計量の表をディレクトリに保存しました: ", file_path))

## T検定

In [ ]:
t.test(age ~ TargetID, data = merged_df7_cln)
t.test(bmiq ~ TargetID, data = merged_df7_cln)
t.test(methw ~ TargetID, data = merged_df7_cln)

In [ ]:
# 年齢(age)について、TargetID（介入群/非介入群）ごとにt検定を実行
t.test(age ~ TargetID, data = matched_data)
t.test(bmiq ~ TargetID, data = matched_data)
t.test(methw ~ TargetID, data = matched_data)

In [ ]:
library(dplyr)
library(broom)

matched_data <- match.data(m.out)

# 比較したい連続変数名のリスト
continuous_vars <- c("age", "bmiq", "methw")

# 2. 各変数に対してt検定を実行し、結果を整形 (tidy) して表を作成
t_test_table_matched <- matched_data %>%
  # TargetIDと対象変数のみを選択
  select(TargetID, all_of(continuous_vars)) %>%
  
  # データを縦長形式に変換
  tidyr::pivot_longer(cols = -TargetID, names_to = "Variable", values_to = "Value") %>%
  
  # 変数ごとにグループ化
  group_by(Variable) %>%
  
  # グループ内でt検定を実行し、broom::tidy()で結果をデータフレームに整形
  do(tidy(t.test(Value ~ TargetID, data = .))) %>%
  
  # 必要な列を選択・整形
  select(Variable, estimate1, estimate2, p.value) %>%
  
  mutate(
    Mean_Case = estimate1, 
    Mean_Control = estimate2,
    P_Value = format.pval(p.value, digits = 3, eps = 0.001, scientific = FALSE)
  ) %>%
  select(Variable, Mean_Case, Mean_Control, P_Value)

# 結果の表を表示
print(t_test_table_matched)
library(dplyr)
library(broom)

# 元のデータセット名 (PSM前のデータ)
data_set <- merged_df7_cln

# 比較したい連続変数名のリスト
continuous_vars <- c("age", "bmiq", "methw")

# 1. 各変数に対してt検定を実行し、結果を整形 (tidy) して結合する
t_test_table <- data_set %>%
  # TargetIDと対象変数のみを選択
  select(TargetID, all_of(continuous_vars)) %>%
  
  # データを縦長形式に変換 
  tidyr::pivot_longer(cols = -TargetID, names_to = "Variable", values_to = "Value") %>%
  
  # 変数ごとにグループ化
  group_by(Variable) %>%
  
  # グループ内でt検定を実行し、broom::tidy()で結果をデータフレームに整形
  do(tidy(t.test(Value ~ TargetID, data = .))) %>%
  
  # 必要な列を選択 
  select(Variable, estimate1, estimate2, p.value) %>%
  
  # P値を整形
  mutate(
    Mean_TargetID_1 = estimate1, 
    Mean_TargetID_0 = estimate2,
    P_Value = format.pval(p.value, digits = 3, eps = 0.001, scientific = FALSE)
  ) %>%
  select(Variable, Mean_TargetID_1, Mean_TargetID_0, P_Value) # 表示する列を整理

# 結果の表を表示
print(t_test_table)

In [ ]:
# 必要なパッケージをロード
library(dplyr)

# マッチング前の結果テーブルにわかりやすい名前を付けます
t_test_table_original <- t_test_table

merged_t_test_table <- t_test_table_original %>%
  # マッチング前の列名を整理・識別しやすくリネーム
  rename(
    Mean_Case_Original = Mean_TargetID_1,
    Mean_Control_Original = Mean_TargetID_0,
    P_Value_Original = P_Value
  ) %>%
  
  # マッチング後の結果を結合
  left_join(
    t_test_table_matched %>%
      # マッチング後の列名を識別しやすくリネーム
      rename(
        Mean_Case_Matched = Mean_Case,
        Mean_Control_Matched = Mean_Control,
        P_Value_Matched = P_Value
      ),
    by = "Variable" # 'Variable' 列をキーとして結合
  ) %>%
  
  # 結果の列順序を整理 (必要に応じて)
  select(
    Variable,
    Mean_Case_Original, Mean_Control_Original, P_Value_Original,
    Mean_Case_Matched, Mean_Control_Matched, P_Value_Matched
  )

# 統合された結果の表を表示
print(merged_t_test_table)

In [ ]:
library(readr)

file_path <- "t_test_comparison.csv"
write_csv(merged_t_test_table, file_path)
cat(paste0("結果の表は '", file_path, "' に保存されました。\n"))

## カイ2乗検定（エラーによりFisher検定に変更）

In [ ]:
#TargetIDごとにカイ二乗検定を実行
chisq.test(table(merged_df7_cln$C_sex, merged_df7_cln$TargetID))
chisq.test(table(merged_df7_cln$C_edu, merged_df7_cln$TargetID))
chisq.test(table(merged_df7_cln$C_dr_yn, merged_df7_cln$TargetID))
chisq.test(table(merged_df7_cln$C_sm_yn, merged_df7_cln$TargetID))
chisq.test(table(merged_df7_cln$C_site, merged_df7_cln$TargetID))

In [ ]:
chisq.test(table(matched_data$C_sex, matched_data$TargetID))
chisq.test(table(matched_data$C_edu, matched_data$TargetID))
chisq.test(table(matched_data$C_dr_yn, matched_data$TargetID))
chisq.test(table(matched_data$C_sm_yn, matched_data$TargetID))
chisq.test(table(matched_data$C_site, matched_data$TargetID))

In [ ]:
library(dplyr)
library(broom)

# 元のデータセット名
data_set <- merged_df7_cln

# 比較したいカテゴリ変数名のリスト
categorical_vars <- c("C_sex", "C_edu", "C_dr_yn", "C_sm_yn", "C_site")

# 1. 各カテゴリ変数に対してカイ二乗検定を実行し、結果を整形 (tidy)
chi_sq_test_table <- data_set %>%
  # TargetIDと対象変数のみを選択
  select(TargetID, all_of(categorical_vars)) %>%

  # データを縦長形式に変換 (一括で適用するため)
  tidyr::pivot_longer(cols = -TargetID, names_to = "Variable", values_to = "Value") %>%

  # 変数ごとにグループ化
  group_by(Variable) %>%

  # グループ内でカイ二乗検定を実行し、broom::tidy()で結果をデータフレームに整形
  do(tidy(chisq.test(.$Value, .$TargetID))) %>%

  # 必要な列を選択・整形
  select(Variable, statistic, parameter, p.value) %>%

  # P値を整形（小数点以下3桁表示、0.001未満は "<0.001"）
  mutate(
    Chi_Square_Stat = round(statistic, 3), # 検定統計量を小数点以下3桁に丸める
    Degrees_of_Freedom = parameter,        # 自由度
    P_Value = format.pval(p.value, digits = 3, eps = 0.001, scientific = FALSE)
  ) %>%
  
  # 表示する列を整理
  select(Variable, Chi_Square_Stat, Degrees_of_Freedom, P_Value)

# 結果の表を表示
print(chi_sq_test_table)

In [ ]:
# 必要なパッケージをロード
library(dplyr)
library(broom)

# マッチング後のデータセット名 (m.outの結果、と仮定)
data_set_matched <- matched_data

# 比較したいカテゴリ変数名のリスト
categorical_vars <- c("C_sex", "C_edu", "C_dr_yn", "C_sm_yn", "C_site")

# 1. 各カテゴリ変数に対してFisherの正確確率検定を実行し、結果を整形
chi_sq_test_table_matched <- data_set_matched %>%
  # TargetIDと対象変数のみを選択
  select(TargetID, all_of(categorical_vars)) %>%
  
  # データを縦長形式に変換
  tidyr::pivot_longer(cols = -TargetID, names_to = "Variable", values_to = "Value") %>%
  
  # 変数ごとにグループ化
  group_by(Variable) %>%
  
  # グループ内でFisherの正確確率検定を実行し、tidy()で結果をデータフレームに整形
  # chisq.test()の期待度数不足の警告を避けるため、fisher.test()を使用
  do(tidy(fisher.test(.$Value, .$TargetID))) %>%
  
  # 必要な列を選択・整形
  select(Variable, p.value) %>%
  
  # P値を整形
  mutate(
    Test_Method_Matched = "Fisher's Exact Test", # 検定方法を明記
    P_Value_Matched = format.pval(p.value, digits = 3, eps = 0.001, scientific = FALSE)
  ) %>%
  
  # 表示する列を整理
  select(Variable, Test_Method_Matched, P_Value_Matched)

# 結果の表を表示
print(chi_sq_test_table_matched)

## Fisher検定（推定P値の出力）

In [ ]:

# 1. C_sex
cat("--- C_sex と TargetID の Fisher の正確確率検定 ---\n")
fisher.test(table(merged_df7_cln$C_sex, merged_df7_cln$TargetID))

# 1. C_edu の Fisher の正確確率検定 (シミュレーションを使用)
cat("--- C_edu と TargetID の Fisher の正確確率検定 (シミュレーション) ---\n")
fisher.test(
    table(merged_df7_cln$C_edu, merged_df7_cln$TargetID), 
    simulate.p.value = TRUE
)

# 2. C_edu の Fisher の正確確率検定 (シミュレーションを使用)
cat("\n--- C_site と TargetID の Fisher の正確確率検定 (シミュレーション) ---\n")
fisher.test(
    table(merged_df7_cln$C_site, merged_df7_cln$TargetID), 
    simulate.p.value = TRUE
)
# 3. C_dr_yn
# --- C_dr_yn の Fisher の正確確率検定 (シミュレーションを使用) ---
cat("--- C_dr_yn と TargetID の Fisher の正確確率検定 (シミュレーション) ---\n")
fisher.test(
    table(merged_df7_cln$C_dr_yn, merged_df7_cln$TargetID), 
    simulate.p.value = TRUE
)

# 4. C_sm_yn
# --- C_dr_yn の Fisher の正確確率検定 (シミュレーションを使用) ---
cat("--- C_sm_yn と TargetID の Fisher の正確確率検定 (シミュレーション) ---\n")
fisher.test(
    table(merged_df7_cln$C_sm_yn, merged_df7_cln$TargetID), 
    simulate.p.value = TRUE
)
# 5. C_site
# --- C_dr_yn の Fisher の正確確率検定 (シミュレーションを使用) ---
cat("--- C_site と TargetID の Fisher の正確確率検定 (シミュレーション) ---\n")
fisher.test(
    table(merged_df7_cln$C_site, merged_df7_cln$TargetID), 
    simulate.p.value = TRUE
)

In [ ]:
# 1. C_sex
cat("--- C_sex (Matched) の Fisher 検定 ---\n")
fisher.test(table(matched_data$C_sex, matched_data$TargetID))

# 2. C_edu
cat("\n--- C_edu (Matched) の Fisher 検定 (シミュレーション) ---\n")
# エラー回避のためシミュレーションを適用
fisher.test(
    table(matched_data$C_edu, matched_data$TargetID), 
    simulate.p.value = TRUE
)

# 3. C_dr_yn
cat("\n--- C_dr_yn (Matched) の Fisher 検定 (シミュレーション) ---\n")
# エラー回避のためシミュレーションを適用
fisher.test(
    table(matched_data$C_dr_yn, matched_data$TargetID), 
    simulate.p.value = TRUE
)

# 4. C_sm_yn
cat("\n--- C_sm_yn (Matched) の Fisher 検定 ---\n")
fisher.test(table(matched_data$C_sm_yn, matched_data$TargetID))

# 5. C_site
cat("\n--- C_site (Matched) の Fisher 検定 (シミュレーション) ---\n")
# エラー回避のためシミュレーションを適用
fisher.test(
    table(matched_data$C_site, matched_data$TargetID), 
    simulate.p.value = TRUE
)

In [ ]:
# 必要なパッケージをロード
library(dplyr)
library(broom)
library(tidyr) # pivot_longerを使用するため

# 元のデータセット名
data_set <- merged_df7_cln

# 比較したいカテゴリ変数名のリスト
categorical_vars <- c("C_sex", "C_edu", "C_dr_yn", "C_sm_yn", "C_site")

# 1. 各カテゴリ変数に対してFisherの正確確率検定 (シミュレーション) を実行し、結果を整形
chi_sq_test_table_original <- data_set %>%
  # TargetIDと対象変数のみを選択
  select(TargetID, all_of(categorical_vars)) %>%
  
  # データを縦長形式に変換
  tidyr::pivot_longer(cols = -TargetID, names_to = "Variable", values_to = "Value") %>%
  
  # 変数ごとにグループ化
  group_by(Variable) %>%
  
  # グループ内でFisherの正確確率検定をシミュレーション付きで実行し、tidy()で結果をデータフレームに整形
  # FEXACT error回避のため、simulate.p.value = TRUE を使用
  do(tidy(fisher.test(.$Value, .$TargetID, simulate.p.value = TRUE))) %>%
  
  # 必要な列を選択・整形
  select(Variable, p.value) %>%
  
  # P値を整形（小数点以下3桁表示、0.001未満は "<0.001"）
  mutate(
    Test_Method_Original = "Fisher's Exact (Simulated)", # 検定方法を明記
    P_Value_Original = format.pval(p.value, digits = 3, eps = 0.001, scientific = FALSE)
  ) %>%
  
  # 表示する列を整理
  select(Variable, Test_Method_Original, P_Value_Original)

# 結果の表を表示
print(chi_sq_test_table_original)
library(dplyr)
library(broom)
library(tidyr) # pivot_longerを使用するため

# マッチング後のデータセット
data_set_matched <- matched_data

# 比較したいカテゴリ変数名のリスト
categorical_vars <- c("C_sex", "C_edu", "C_dr_yn", "C_sm_yn", "C_site")

# 1. 各カテゴリ変数に対してFisherの正確確率検定 (シミュレーション) を実行し、結果を整形
chi_sq_test_table_matched <- data_set_matched %>%
  # TargetIDと対象変数のみを選択
  select(TargetID, all_of(categorical_vars)) %>%
  
  # データを縦長形式に変換
  tidyr::pivot_longer(cols = -TargetID, names_to = "Variable", values_to = "Value") %>%
  
  # 変数ごとにグループ化
  group_by(Variable) %>%
  
  # グループ内でFisherの正確確率検定をシミュレーション付きで実行し、tidy()で結果をデータフレームに整形
  # FEXACT error回避のため、simulate.p.value = TRUE を使用します
  do(tidy(fisher.test(.$Value, .$TargetID, simulate.p.value = TRUE))) %>%
  
  # 必要な列を選択・整形
  select(Variable, p.value) %>%
  
  # P値を整形（小数点以下3桁表示、0.001未満は "<0.001"）
  mutate(
    Test_Method_Matched = "Fisher's Exact (Simulated)", 
    P_Value_Matched = format.pval(p.value, digits = 3, eps = 0.001, scientific = FALSE)
  ) %>%
  
  # 表示する列を整理
  select(Variable, Test_Method_Matched, P_Value_Matched)

# 結果の表を表示
print(chi_sq_test_table_matched)

In [ ]:
# 必要なパッケージをロード
library(dplyr)

# 統合するためのキーとなる列名 'Variable' を使用して、2つの表を左結合
merged_chi_sq_table <- chi_sq_test_table_original %>%
  
  # マッチング後の結果を結合
  left_join(chi_sq_test_table_matched, by = "Variable") %>%
  
  # 結果の列順序を整理
  select(
    Variable, 
    Test_Method_Original, P_Value_Original, 
    Test_Method_Matched, P_Value_Matched
  )

# 統合された結果の表を表示
print(merged_chi_sq_table)

In [ ]:
library(readr) 
file_path <- "chi_sq_fisher_comparison.csv"
write_csv(merged_chi_sq_table, file_path)
cat(paste0("カテゴリ変数の統合結果は '", file_path, "' に保存されました。\n"))

## K-S検定

In [ ]:
#分布がTargetID（介入群/非介入群）で等しいかをKS検定で確認
ks.test(merged_df7_cln$age[merged_df7_cln$TargetID == 1], 
        merged_df7_cln$age[merged_df7_cln$TargetID == 0])   
ks.test(merged_df7_cln$bmiq[merged_df7_cln$TargetID == 1],  
        merged_df7_cln$bmiq[merged_df7_cln$TargetID == 0])   
ks.test(merged_df7_cln$methw[merged_df7_cln$TargetID == 1],  
        merged_df7_cln$methw[merged_df7_cln$TargetID == 0])  
ks.test(matched_data$age[matched_data$TargetID == 1],  
        matched_data$age[matched_data$TargetID == 0])   
ks.test(matched_data$bmiq[matched_data$TargetID == 1],  
        matched_data$bmiq[matched_data$TargetID == 0])   
ks.test(matched_data$methw[matched_data$TargetID == 1],  
        matched_data$methw[matched_data$TargetID == 0])   

In [ ]:
library(dplyr)
library(broom)
library(tidyr)

# 比較したい連続変数名のリスト (T検定と同じ)
continuous_vars <- c("age", "bmiq", "methw")
ks_test_table_original <- merged_df7_cln %>%
  # TargetIDと対象変数のみを選択
  select(TargetID, all_of(continuous_vars)) %>%
  
  # データを縦長形式に変換
  tidyr::pivot_longer(cols = -TargetID, names_to = "Variable", values_to = "Value") %>%
  
  # 変数ごとにグループ化
  group_by(Variable) %>%
  
  # グループ内でKS検定を実行し、broom::tidy()で結果をデータフレームに整形
  do(tidy(ks.test(.$Value[.$TargetID == 1], 
                  .$Value[.$TargetID == 0]))) %>%
  
  # 必要な列を選択・整形
  select(Variable, statistic, p.value) %>%
  
  mutate(
    KS_Stat_Original = round(statistic, 4), # KS統計量 (D値) を小数点以下4桁に丸める
    P_Value_Original = format.pval(p.value, digits = 3, eps = 0.001, scientific = FALSE)
  ) %>%
  
  select(Variable, KS_Stat_Original, P_Value_Original)
ks_test_table_matched <- matched_data %>%
  # TargetIDと対象変数のみを選択
  select(TargetID, all_of(continuous_vars)) %>%
  
  # データを縦長形式に変換
  tidyr::pivot_longer(cols = -TargetID, names_to = "Variable", values_to = "Value") %>%
  
  # 変数ごとにグループ化
  group_by(Variable) %>%
  
  # グループ内でKS検定を実行し、tidy()で結果をデータフレームに整形
  do(tidy(ks.test(.$Value[.$TargetID == 1], 
                  .$Value[.$TargetID == 0]))) %>%
  
  # 必要な列を選択・整形
  select(Variable, statistic, p.value) %>%
  
  mutate(
    KS_Stat_Matched = round(statistic, 4), 
    P_Value_Matched = format.pval(p.value, digits = 3, eps = 0.001, scientific = FALSE)
  ) %>%
  
  select(Variable, KS_Stat_Matched, P_Value_Matched)
merged_ks_test_table <- ks_test_table_original %>%
  # Variableをキーとしてマッチング後の結果を結合
  left_join(ks_test_table_matched, by = "Variable") %>%
  
  # 結果の列順序を整理
  select(
    Variable, 
    KS_Stat_Original, P_Value_Original, 
    KS_Stat_Matched, P_Value_Matched
  )

# 統合された結果の表を表示
print(merged_ks_test_table)

In [ ]:
library(readr) 
file_path <- "ks_test_distribution_comparison.csv"
write_csv(merged_ks_test_table, file_path)
cat(paste0("KS検定の統合結果は '", file_path, "' に保存されました。✅\n"))

## Wilcoxon検定

In [ ]:
# 加工前のデータ（merged_df7_cln）を使用し、ageの中央値をTargetIDで比較
wilcox.test(age ~ TargetID, data = merged_df7_cln)
wilcox.test(bmiq ~ TargetID, data = merged_df7_cln)
wilcox.test(methw ~ TargetID, data = merged_df7_cln)
# TargetIDごとにWilcoxon順位和検定を実行
# (データが正規分布しない場合のバランス評価やアウトカム比較に使用)
wilcox.test(age ~ TargetID, data = matched_data)
wilcox.test(bmiq ~ TargetID, data = matched_data)
wilcox.test(methw ~ TargetID, data = matched_data)

In [ ]:
# 必要なパッケージをロード
library(dplyr)
library(broom)
library(tidyr) # pivot_longerを使用

# 比較したい連続変数名のリスト
continuous_vars <- c("age", "bmiq", "methw")
wilcox_test_table_original <- merged_df7_cln %>%
  # TargetIDと対象変数のみを選択
  select(TargetID, all_of(continuous_vars)) %>%
  
  # データを縦長形式に変換
  tidyr::pivot_longer(cols = -TargetID, names_to = "Variable", values_to = "Value") %>%
  
  # 変数ごとにグループ化
  group_by(Variable) %>%
  
  # グループ内でWilcoxon順位和検定を実行し、tidy()で結果をデータフレームに整形
  do(tidy(wilcox.test(Value ~ TargetID, data = .))) %>%
  
  # 必要な列を選択・整形
  select(Variable, statistic, p.value) %>%
  
  mutate(
    # WilcoxonのW統計量をそのまま格納
    W_Stat_Original = statistic, 
    P_Value_Original = format.pval(p.value, digits = 3, eps = 0.001, scientific = FALSE)
  ) %>%
  
  select(Variable, W_Stat_Original, P_Value_Original)

wilcox_test_table_matched <- matched_data %>%
  # TargetIDと対象変数のみを選択
  select(TargetID, all_of(continuous_vars)) %>%
  
  # データを縦長形式に変換
  tidyr::pivot_longer(cols = -TargetID, names_to = "Variable", values_to = "Value") %>%
  
  # 変数ごとにグループ化
  group_by(Variable) %>%
  
  # グループ内でWilcoxon順位和検定を実行し、tidy()で結果をデータフレームに整形
  do(tidy(wilcox.test(Value ~ TargetID, data = .))) %>%
  
  # 必要な列を選択・整形
  select(Variable, statistic, p.value) %>%
  
  mutate(
    W_Stat_Matched = statistic, 
    P_Value_Matched = format.pval(p.value, digits = 3, eps = 0.001, scientific = FALSE)
  ) %>%
  
  select(Variable, W_Stat_Matched, P_Value_Matched)

merged_wilcox_test_table <- wilcox_test_table_original %>%
  # Variableをキーとしてマッチング後の結果を結合
  left_join(wilcox_test_table_matched, by = "Variable") %>%
  
  # 結果の列順序を整理
  select(
    Variable, 
    W_Stat_Original, P_Value_Original, 
    W_Stat_Matched, P_Value_Matched
  )

# 統合された結果の表を表示
print(merged_wilcox_test_table)

In [ ]:
library(readr) 
file_path <- "wilcoxon_test_comparison.csv"
write_csv(merged_wilcox_test_table, file_path)
cat(paste0("Wilcoxon検定の統合結果は '", file_path, "' に保存されました。✅\n"))